<a href="https://colab.research.google.com/github/cu-applied-math/appm-4600-numerics/blob/lab_solutions/Labs/Lab12_LeastSquares_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 12: Least Squares (SOLUTIONS)

This lab focuses on the problem
$$\min_{\vec{x}\in\mathbb{R}^m}\;\|A\vec{x}-\vec{b}\|_2$$
where $A\in\mathbb{R}^{m\times n}$, and $m\ge n$ and $\text{rank}(A)=n$.

We'll make problems where we can check our answer, first creating a "true" $\vec{x}_{\text{true}}$, then set $\vec{b}=A \vec{x}_{\text{true}}$ for a given $A$ matrix, and your job is to solve for
$\vec{x}_{\text{estimate}}=\text{argmin}_{\vec{x}}\|A\vec{x}-\vec{b}\|_2$

#### Learning objectives
- Learn how to solve a least squares problem "from scratch" (well, partially from scratch)
- Understand pros and cons of different methods
  - Which ones require pivoting?
  - Which ones are faster?
  - Which ones are faster on multiple cores or a GPU?  (i.e., more parallelizable)
  - Which ones are more accurate?
  - Does this depend on conditioning of $A$? On its shape? (i.e., almost square vs very tall and skinny)

#### Tasks
1. Write at least **four** distinct implementations to solve the least-squares problem (the solutions have about implementations). The implementations can be similar to each other, but each variant should be non-trivial.  You **are** allowed to use linear algebra library functions, such as:
    - `scipy.linalg.inverse`
    - `scipy.linalg.solve` and all its variants (i.e., the different options for `assume_a`)
    - `scipy.linalg.qr` and all its variants (i.e., pivoting or not)
    - `scipy.linalg.solve_triangular` for forward/back substitution
    - `scipy.linalg.qr_multiply`
    - `scipy.linalg.svd`
    - `scipy.linalg.lstsq` and all its variants (i.e., the different options for `lapack_driver`). Since this routine does everything for you, you may only count it as **one** of your implementations (that is, if you do different options for `lapack_driver`, those don't count as separate implementations).
2. Use your implementations
    - Try "problem 0", a small easy problem that is good for checking whether your code works
    - Then try "problem 1" and "problem 2" which are larger and less well-conditioned. How well do the implementations work?  
        - **Time** your code
        - Also **record the error** in the $\ell_\infty$ norm, i.e., $\|\vec{x_{true}} - \vec{x_{est}}\|_\infty$
3. If you have time, try an implementation on the GPU. *We highly recommend that this part of the lab is done on google colab*
    - Use pytorch (already installed on colab)
    - Select a GPU runtime
    - See the scaffolded code below

#### Deliverables
1. Code for at least 4 implementations, and...
2. at least one sentence (or more) about some pros and cons of different methods.

Put both these deliverables into the same PDF and upload to Canvas

*APPM 4600. Copyright 2025 Department of Applied Mathematics, University of Colorado Boulder. Released under a BSD 3-clause license*

In [13]:
import numpy as np
import scipy.linalg as sla

## Part 1: write at least **four** implementations to solve the least-squares problem



In [15]:
# Problem 0: use this for debugging
problem_label = 'Problem 0'
rng     = np.random.default_rng(12345)
n       = 20
m       = 40
A       = rng.standard_normal( (m,n), dtype=np.double )
xTrue   = rng.standard_normal(n)
b       = A@xTrue
print(f'Matrix is size {m} x {n}')
print(f'Condition number of A is {np.linalg.cond(A):.2e}')

Matrix is size 40 x 20
Condition number of A is 3.78e+00


In [ ]:
# Your solutions

x = ... TODO ...

err   = np.linalg.norm(x-xTrue,np.inf)
print(f'Error is {err:.2e}')

In [14]:
# SOLUTIONS

all_methods = []

def normal_equations_inverse(A,b):
    return np.linalg.inv( A.T @ A ) @ (A.T@b )
normal_equations_inverse.label = 'Solve normal equations using the inverse'
all_methods.append( normal_equations_inverse )

def normal_equations_solve1(A,b):
    return sla.solve( A.T@A, A.T@b)
normal_equations_solve1.label='Solve normal equations using LU'
all_methods.append( normal_equations_solve1 )

def normal_equations_solve2(A,b):
    return sla.solve( A.T@A, A.T@b, assume_a='symmetric')
normal_equations_solve2.label='Solve normal equations using LDL'
all_methods.append( normal_equations_solve2 )

def normal_equations_solve3(A,b):
    # No pivoting needed! Should be faster
    return sla.solve( A.T@A, A.T@b, assume_a='positive definite')
normal_equations_solve3.label='Solve normal equations using Cholesky'
all_methods.append( normal_equations_solve3 )

def QR_nopivoting(A,b):
    # Q,R = numpy.linalg.qr( A, mode="reduced")
    Q,R = sla.qr( A, mode="economic") # scipy version slightly different
    Qtb = Q.T@b
    return sla.solve_triangular(R, Qtb )
QR_nopivoting.label='Solve via thin QR, no pivoting'
all_methods.append( QR_nopivoting )

def QR_pivoting(A,b):
    Q,R,pivots = sla.qr( A, mode="economic",pivoting=True)
    # so A[:, P] = Q @ R
    Qtb = Q.T@b
    x   = np.empty(A.shape[1])
    x[pivots]   = sla.solve_triangular(R, Qtb )
    return x
QR_pivoting.label='Solve via thin QR, with pivoting'
all_methods.append( QR_pivoting )

def QR_nopivoting_fast(A,b):
    Qtb, R = sla.qr_multiply( A, b, mode="right", pivoting=False )
    return sla.solve_triangular(R, Qtb )
QR_nopivoting_fast.label='Solve via thin QR, no pivoting, keeping Q implicit'
all_methods.append( QR_nopivoting_fast )

def QR_pivoting_fast(A,b):
    Qtb, R, pivots = sla.qr_multiply( A, b, mode="right", pivoting=True )
    x   = np.empty(A.shape[1])
    x[pivots]   = sla.solve_triangular(R, Qtb )
    return x
QR_pivoting_fast.label='Solve via thin QR, with pivoting, keeping Q implicit'
all_methods.append( QR_pivoting_fast )

def SVD(A,b):
  U,S,Vh = sla.svd(A,full_matrices=False)
  return Vh.T @ (np.diag(1/S) @ (U.T @ b) )
SVD.label='Solve via SVD'
all_methods.append( SVD )

def builtin_leastsquares(A,b):
    """ least squares using SVD, divide and conquer """
    # np.linalg.lstsq(A,b, rcond=None)[0] # numpy slightly different than scipy
    # Default driver is gelsd
    # https://www.netlib.org/lapack/explore-html/d9/d67/group__gelsd.html
    return sla.lstsq(A,b)[0]
builtin_leastsquares.label='Builtin least-squares solver, gelsd driver'
all_methods.append( builtin_leastsquares )

def builtin_leastsquares2(A,b):
    """least squares using complete orthogonal factor """
    # https://www.netlib.org/lapack/explore-html/dc/d8b/group__gelsy.html
    return sla.lstsq(A,b,lapack_driver='gelsy')[0]
builtin_leastsquares2.label='Builtin least-squares solver, gelsy driver'
all_methods.append( builtin_leastsquares2 )

def builtin_leastsquares3(A,b):
    """ least squares using SVD, QR iteration """
    # https://www.netlib.org/lapack/explore-html/da/d55/group__gelss.html
    return sla.lstsq(A,b,lapack_driver='gelss')[0]
builtin_leastsquares3.label='Builtin least-squares solver, gelss driver'
all_methods.append( builtin_leastsquares3 )

## Part 2: Try solving some larger problems
- Record the **error** in the infinity norm, and...
- Record the **time** it takes

To measure time, in jupyter/colab/iPython, you can use a line like

`%time x = myFunction(...)`

and it will time how long it takes `myFunction(...)` to run


What **observations** can you make?

Note that if you use `%time`, it will return both the:
1. CPU time, which is the sum of how long all cores on the computer spend. So if you have 4 cores, and each core takes 3 seconds (and is fully utilized), the CPU time is 12 seconds.
2. "Wall" time, which refers to "wall clock" time. This is the actual time elapsed from when you start to when you stop.

If the CPU time is 12 seconds and the wall time is 3 seconds, this indicates that the code ran in parallel.

In [15]:
# Problem 1
problem_label = 'Problem 1'
rng     = np.random.default_rng(12345)
# n       = int(5e3)
n       = int(1e3)
m       = n+1
A       = rng.standard_normal( (m,n), dtype=np.single )
xTrue   = rng.standard_normal(n)
b       = A@xTrue
print(f'Matrix is size {m} x {n}')
print(f'Condition number of A is {np.linalg.cond(A):.2e}') # takes a bit of time to run! It's about 4.6e3 if n=5e3

Matrix is size 1001 x 1000
Condition number of A is 1.24e+03


In [18]:
print(problem_label)
for method in all_methods:
    print('=== ' + method.label + ' ===')
    %time x = method(A,b)
    err   = np.linalg.norm(x-xTrue,np.inf)
    print(f'and l_inf error is {err:.2e}')
    print('')

Problem 1
=== Solve normal equations using the inverse ===
CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 227 ms
and l_inf error is 5.21e-03

=== Solve normal equations using LU ===
CPU times: user 332 ms, sys: 0 ns, total: 332 ms
Wall time: 168 ms
and l_inf error is 4.22e-03

=== Solve normal equations using LDL ===
CPU times: user 377 ms, sys: 1.98 ms, total: 379 ms
Wall time: 196 ms
and l_inf error is 4.22e-03

=== Solve normal equations using Cholesky ===
CPU times: user 166 ms, sys: 10 µs, total: 166 ms
Wall time: 99.7 ms
and l_inf error is 4.22e-03

=== Solve via thin QR, no pivoting ===
CPU times: user 249 ms, sys: 1 ms, total: 250 ms
Wall time: 133 ms
and l_inf error is 5.79e-05

=== Solve via thin QR, with pivoting ===
CPU times: user 416 ms, sys: 28 µs, total: 416 ms
Wall time: 210 ms
and l_inf error is 3.90e-05

=== Solve via thin QR, no pivoting, keeping Q implicit ===
CPU times: user 228 ms, sys: 35 µs, total: 228 ms
Wall time: 114 ms
and l_inf error is 7.07e-

In [24]:
# Problem 2
problem_label = 'Problem 2'
rng = np.random.default_rng(12345)

# n   = int(2e3)
n   = int(1e3)
n2  = int(n/2)
# m   = int(1e4)
m   = int(5e3)
A       = rng.standard_normal( (m,n2) )
A       = np.hstack(  (A,A+1e-3*rng.standard_normal( (m,n2) )), dtype=np.single )
xTrue   = rng.standard_normal(n)
b       = A@xTrue
print(f'Matrix is size {m} x {n}')
print(f'Condition number of A is {np.linalg.cond(A):.2e}') # takes a bit of time to run! It's about 4.1e4 when m=1e4, 4.1e3 when m=5000

Matrix is size 5000 x 1000
Condition number of A is 4.14e+03


In [25]:
print(problem_label)
for method in all_methods:
    print('=== ' + method.label + ' ===')
    %time x = method(A,b)
    err   = np.linalg.norm(x-xTrue,np.inf)
    print(f'and l_inf error is {err:.2e}')
    print('')

Problem 2
=== Solve normal equations using the inverse ===
CPU times: user 535 ms, sys: 9.91 ms, total: 545 ms
Wall time: 563 ms
and l_inf error is 4.65e+01

=== Solve normal equations using LU ===
CPU times: user 557 ms, sys: 9.65 ms, total: 566 ms
Wall time: 438 ms
and l_inf error is 4.42e+01

=== Solve normal equations using LDL ===
CPU times: user 532 ms, sys: 11.7 ms, total: 544 ms
Wall time: 438 ms
and l_inf error is 4.42e+01

=== Solve normal equations using Cholesky ===
CPU times: user 452 ms, sys: 7.64 ms, total: 460 ms
Wall time: 346 ms
and l_inf error is 4.42e+01

=== Solve via thin QR, no pivoting ===
CPU times: user 1.2 s, sys: 7.89 ms, total: 1.21 s
Wall time: 653 ms
and l_inf error is 6.08e-04

=== Solve via thin QR, with pivoting ===
CPU times: user 2.86 s, sys: 11 ms, total: 2.87 s
Wall time: 1.5 s
and l_inf error is 9.43e-04

=== Solve via thin QR, no pivoting, keeping Q implicit ===
CPU times: user 776 ms, sys: 62 µs, total: 776 ms
Wall time: 393 ms
and l_inf error i

## Part 3: Try it on the GPU

Restart the colab runtime and select a GPU runtime (or try a TPU!).  If you register for the Colab "pro" account (free for students and educators!) you'll get access to more/faster GPUs.

We'll use PyTorch for our GPU computing.  You *can* run this just on the CPU also.  Some code is specific for either CPU or GPU, but most code runs on both.

PyTorch has its own `torch.linalg` library, which is similar to (but not the same) as `numpy.linalg` and `scipy.linalg`.  It has GPU implementations, but lacks some of the fancier features that scipy has, and overall has fewer methods.

For timing, you can still use `%time` for a single line, or `%%time` to time an entire cell. However, you need to make sure to call `torch.cuda.synchronize()` within the timed code, otherwise it won't time it properly (it will think that the code is done running when it's not really done).

In [1]:
import torch
import torch.linalg as tsla
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print('Device is ', device)

Device is  cpu


In [ ]:
# Problem 0: use this for debugging
problem_label = 'Problem 0'

torch.manual_seed(1234)
n       = 20
m       = 40
A       = torch.randn(m,n,device=device) # by default it's single precision
xTrue   = torch.randn(n,1,device=device)
b       = A@xTrue
print(f'Matrix is size {m} x {n}, device is {device}')
print(f'Condition number of A is {tsla.cond(A):.2e}') # 1e3

In [2]:
# SOLUTIONS

all_methods_torch = []

def normal_equations_inverse(A,b):
    x = tsla.inv( A.T @ A ) @ (A.T@b )
    if b.is_cuda: torch.cuda.synchronize()
    return x
normal_equations_inverse.label = 'Solve normal equations using the inverse'
all_methods_torch.append( normal_equations_inverse )

def normal_equations_solve1(A,b):
    x = tsla.solve( A.T@A, A.T@b)
    if b.is_cuda: torch.cuda.synchronize()
    return x
normal_equations_solve1.label='Solve normal equations using LU'
all_methods_torch.append( normal_equations_solve1 )

def normal_equations_solve2(A,b):
    if b.is_cuda:
        LU, pivots = torch.linalg.lu_factor(A.T@A, pivot=False)
        x = torch.linalg.lu_solve( LU, pivots, A.T@b)
        torch.cuda.synchronize()
        return x
    else:
        print('lu_factor without pivoting not implemented on CPU, returning zero soln')
        return torch.zeros(A.shape[1],1)
normal_equations_solve2.label='Solve normal equations using LU, no pivoting'
all_methods_torch.append( normal_equations_solve2 )

def normal_equations_solve3(A,b):
    L = tsla.cholesky(A.T@A) # L@L.T
    y = tsla.solve_triangular(L, A.T@b , upper=False)
    x = tsla.solve_triangular(L.T, y, upper=True )
    if b.is_cuda: torch.cuda.synchronize()
    return x
normal_equations_solve3.label='Solve normal equations using Cholesky'
all_methods_torch.append( normal_equations_solve3 )


def QR_nopivoting(A,b):
    Q,R = tsla.qr( A, mode="reduced")
    Qtb = Q.T@b
    x = tsla.solve_triangular(R, Qtb, upper=True )
    if b.is_cuda: torch.cuda.synchronize()
    return x
QR_nopivoting.label='Solve via thin QR, no pivoting'
all_methods_torch.append( QR_nopivoting )

def builtin_leastsquares(A,b):
    x =  tsla.lstsq(A,b,driver='gels')[0]
    if b.is_cuda: torch.cuda.synchronize()
    return x
builtin_leastsquares.label='Builtin least-squares solver'
all_methods_torch.append( builtin_leastsquares )

In [10]:
problem_label = 'Problem 1'
torch.manual_seed(1234)
n       = int(5e3)
m       = n+10
A       = torch.randn(m,n,device=device) # by default it's single precision
xTrue   = torch.randn(n,1,device=device)
b       = A@xTrue
print(f'Matrix is size {m} x {n}, device is {device}')
print(f'Condition number of A is {tsla.cond(A):.2e}') # 1e3

Matrix is size 5010 x 5000, device is cpu
Condition number of A is 1.73e+03


In [8]:
problem_label = 'Problem 2'
torch.manual_seed(0)

n   = int(2e3)
n2  = int(n/2)
m   = int(1e4)
A       = torch.randn(m,n2,device=device) # by default it's single precision
A       = torch.hstack(  (A,A+1e-2*torch.randn(m,n2,device=device) ) )
xTrue   = torch.randn(n,1,device=device)
b       = A@xTrue
print(f'Matrix is size {m} x {n}')
print(f'Condition number of A is {tsla.cond(A):.2e}') # 4e2

Matrix is size 10000 x 2000
Condition number of A is 4.14e+02


In [11]:
print(problem_label)
print(f'device is {device}')
for method in all_methods_torch:
    print('=== ' + method.label + ' ===')
    %time x = method(A,b)
    err   = torch.norm(x-xTrue,float('inf'))
    # err   = torch.norm(x-xTrue)
    print(f'and l_inf error is {err:.2e}')
    print('')

Problem 1
device is cpu
=== Solve normal equations using the inverse ===
CPU times: user 8.94 s, sys: 180 ms, total: 9.11 s
Wall time: 2.31 s
and l_inf error is 1.87e-01

=== Solve normal equations using LU ===
CPU times: user 5.55 s, sys: 228 ms, total: 5.77 s
Wall time: 1.44 s
and l_inf error is 1.02e-01

=== Solve normal equations using LU, no pivoting ===
lu_factor without pivoting not implemented on CPU, returning zero soln
CPU times: user 96 µs, sys: 0 ns, total: 96 µs
Wall time: 99.2 µs
and l_inf error is 3.75e+00

=== Solve normal equations using Cholesky ===
CPU times: user 5.78 s, sys: 123 ms, total: 5.9 s
Wall time: 1.48 s
and l_inf error is 1.55e-01

=== Solve via thin QR, no pivoting ===
CPU times: user 8.84 s, sys: 186 ms, total: 9.02 s
Wall time: 2.32 s
and l_inf error is 6.70e-05

=== Builtin least-squares solver ===
CPU times: user 3.84 s, sys: 57 ms, total: 3.9 s
Wall time: 976 ms
and l_inf error is 1.03e-04



## CPU output was:
#### Problem 1
```
Problem 1
device is cpu
=== Solve normal equations using the inverse ===
CPU times: user 8.94 s, sys: 180 ms, total: 9.11 s
Wall time: 2.31 s
and l_inf error is 1.87e-01

=== Solve normal equations using LU ===
CPU times: user 5.55 s, sys: 228 ms, total: 5.77 s
Wall time: 1.44 s
and l_inf error is 1.02e-01

=== Solve normal equations using Cholesky ===
CPU times: user 5.78 s, sys: 123 ms, total: 5.9 s
Wall time: 1.48 s
and l_inf error is 1.55e-01

=== Solve via thin QR, no pivoting ===
CPU times: user 8.84 s, sys: 186 ms, total: 9.02 s
Wall time: 2.32 s
and l_inf error is 6.70e-05

=== Builtin least-squares solver ===
CPU times: user 3.84 s, sys: 57 ms, total: 3.9 s
Wall time: 976 ms
and l_inf error is 1.03e-04
```
#### Problem 2
```
Problem 2
device is cpu
=== Solve normal equations using the inverse ===
CPU times: user 1.69 s, sys: 13.9 ms, total: 1.7 s
Wall time: 429 ms
and l_inf error is 1.08e-01

=== Solve normal equations using LU ===
CPU times: user 1.51 s, sys: 26.1 ms, total: 1.54 s
Wall time: 398 ms
and l_inf error is 7.40e-02

=== Solve normal equations using Cholesky ===
CPU times: user 1.31 s, sys: 25.9 ms, total: 1.34 s
Wall time: 334 ms
and l_inf error is 1.07e-01

=== Solve via thin QR, no pivoting ===
CPU times: user 3.46 s, sys: 103 ms, total: 3.56 s
Wall time: 892 ms
and l_inf error is 1.76e-04

=== Builtin least-squares solver ===
CPU times: user 1.84 s, sys: 69 ms, total: 1.9 s
Wall time: 476 ms
and l_inf error is 1.10e-04
```

## GPU (L40) output
#### Problem 1
```
Problem 1
device is cuda
=== Solve normal equations using the inverse ===
CPU times: user 79.8 ms, sys: 0 ns, total: 79.8 ms
Wall time: 79.5 ms
and l_inf error is 8.65e-02

=== Solve normal equations using LU ===
CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 56 ms
and l_inf error is 3.42e-02

=== Solve normal equations using LU, no pivoting ===
CPU times: user 39.8 ms, sys: 0 ns, total: 39.8 ms
Wall time: 39.8 ms
and l_inf error is 4.36e-02

=== Solve normal equations using Cholesky ===
CPU times: user 34.9 ms, sys: 859 µs, total: 35.8 ms
Wall time: 35.2 ms
and l_inf error is 4.08e-02

=== Solve via thin QR, no pivoting ===
CPU times: user 66 ms, sys: 27 ms, total: 93 ms
Wall time: 93 ms
and l_inf error is 9.88e-05

=== Builtin least-squares solver ===
CPU times: user 64.3 ms, sys: 13 ms, total: 77.3 ms
Wall time: 77.3 ms
and l_inf error is 9.10e-05
```
#### Problem 2
```
Problem 2
device is cuda
=== Solve normal equations using the inverse ===
CPU times: user 14.3 ms, sys: 39 µs, total: 14.4 ms
Wall time: 14.1 ms
and l_inf error is 1.52e-01

=== Solve normal equations using LU ===
CPU times: user 10.5 ms, sys: 0 ns, total: 10.5 ms
Wall time: 10.5 ms
and l_inf error is 1.17e-01

=== Solve normal equations using LU, no pivoting ===
CPU times: user 6.84 ms, sys: 0 ns, total: 6.84 ms
Wall time: 6.84 ms
and l_inf error is 3.07e-01

=== Solve normal equations using Cholesky ===
CPU times: user 8.02 ms, sys: 0 ns, total: 8.02 ms
Wall time: 8.02 ms
and l_inf error is 3.21e-01

=== Solve via thin QR, no pivoting ===
CPU times: user 50.4 ms, sys: 0 ns, total: 50.4 ms
Wall time: 50.4 ms
and l_inf error is 1.44e-04

=== Builtin least-squares solver ===
CPU times: user 44.7 ms, sys: 0 ns, total: 44.7 ms
Wall time: 44.7 ms
and l_inf error is 8.51e-05
```